<a href="https://colab.research.google.com/github/marshallarnold/Fall-Detection-Image-Classifier/blob/main/falldetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/valid'

train_datagen = ImageDataGenerator(rescale=1/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255)

valid_datagen = ImageDataGenerator(rescale=1/255)

batch_size = 32
class_mode = 'binary'

train_data = train_datagen.flow_from_directory(train_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode)
test_data = test_datagen.flow_from_directory(test_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode, shuffle = False)
valid_data = valid_datagen.flow_from_directory(valid_dir, batch_size = batch_size,
            target_size = (224,224), class_mode = class_mode)

Found 909 images belonging to 2 classes.
Found 51 images belonging to 2 classes.
Found 101 images belonging to 2 classes.


In [26]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape = (224,224,3)),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

In [28]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_data, epochs=10, validation_data=valid_data, callbacks=[early_stop])

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.8694 - loss: 0.3167 - val_accuracy: 1.0000 - val_loss: 0.0784
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.9799 - loss: 0.0482 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9936 - loss: 0.0253 - val_accuracy: 1.0000 - val_loss: 7.6561e-05
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9942 - loss: 0.0135 - val_accuracy: 1.0000 - val_loss: 1.3007e-06
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9997 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 3.7471e-07
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9996 - loss: 0.0062 - val_accuracy: 1.0000 - val_loss: 5.6261e-07
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9993 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 4.6528e-07
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.9981 - loss: 0.0056 - val_accuracy

In [30]:
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 222ms/step - accuracy: 1.0000 - loss: 3.7227e-07
Test accuracy: 1.0


In [31]:
from google.colab import drive
import os
drive.mount('/content/drive')
model_save_path = '/content/drive/MyDrive/models/fall_detection_model.h5'
os.makedirs('/content/drive/MyDrive/models', exist_ok=True)
model.save(model_save_path)
print(f"Model saved to {model_save_path}")
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/models/fall_detection_model.h5')

Mounted at /content/drive


Model saved to /content/drive/MyDrive/models/fall_detection_model.h5
